In [ ]:
!pip install pycuda
!pip install scikit-cuda
!pip install hickle

In [ ]:
import numpy as np
import sys
import os
import cv2
from sklearn.utils import check_random_state
from sklearn.metrics import mean_squared_error as mse
from sklearn.preprocessing import normalize, MinMaxScaler
from sklearn.metrics.pairwise import polynomial_kernel
import math
import time
from sklearn.decomposition import PCA
import tensorflow as tf
import pycuda.autoinit
import pycuda.driver as cuda
import pycuda.gpuarray as gpua
import pycuda.curandom
import pycuda.driver as drv
from pycuda.compiler import SourceModule
from pycuda import cumath
import hickle as hkl
import skcuda
import skcuda.linalg as linalg
linalg.init()



physical_devices = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

/usr/local/lib/python3.7/dist-packages/skcuda/cublas.py:284: UserWarning: creating CUBLAS context to get version number
  warnings.warn('creating CUBLAS context to get version number')


In [ ]:
# Matrix Inverse
def matrix_inverse(A, cuda = True, coefficient = None):
    m, n = A.shape
    transpose = False
    pinv = None
    
#     A is m-by-n and the rank of A is equal to m (m ≤ n) then A has right inverse A_P = At * inv(A * At)
#     A is m-by-n and the rank of A is equal to n (n ≤ m) then A has left inverse A_P = A * inv(At * A)

    if m > n:
        A = np.ascontiguousarray(A.T)
        transpose = True
# By default this calculates the right inverse
    At = np.ascontiguousarray(A.T)
    A = np.ascontiguousarray(A)
    if cuda:
        A_gpu = gpua.to_gpu(A)
        At_gpu = gpua.to_gpu(At)
        
        if coefficient is None:
            inv = linalg.inv(linalg.dot(A_gpu, At_gpu))
        else:
            inv = linalg.inv( gpua.to_gpu(np.identity(A.shape[0])/coefficient) + linalg.dot(A_gpu, At_gpu))
        pinv = linalg.dot(At_gpu, inv).get()
    else:
        inv = np.linalg.inv(A.dot(At))
        pinv = At.dot(inv)
        
    if transpose:
        pinv = pinv.T
        
    return pinv

def exp(x):
    x_gpu = gpua.to_gpu(x)
    x_exp = cumath.exp(x_gpu).get()
    return x_exp

#maxpooling
def maxpool(A,B):
  C=A
  for i in range(len(C)):
    for j in range(len(C[0])):
      if(A[i][j] > B[i][j]):
        C[i][j] = A[i][j]
      else:
        C[i][j] = B[i][j]
  return C

#avgpooling
def avgpool(A,B):
  C=A
  for i in range(len(C)):
    for j in range(len(C[0])):
      C[i][j] = (A[i][j]+B[i][j])/2
  return C


In [ ]:
def subnetwrork(P, T, TV_P, TV_T, NumberofHiddenNeurons, C, kkkk, sn, name, chnl):
    fdafe=0
    fdafe1=0
    asin = np.vectorize(math.asin)
    sin = np.vectorize(math.sin)
    saveT=T
    OrgT=T
    OrgP=P
    OrgTT=TV_T
    NumberofTrainingData=len(P[1])
    NumberofTestingData=len(TV_P[1])
    NumberofInputNeurons=len(P);
    NumberofOutputNeurons=len(np.unique(T));
    for subnetwork in range(1,sn+1):
        for j in range(1,kkkk+1):
            if(j==1):
                BiasofHiddenNeurons1 = np.random.rand(NumberofHiddenNeurons)
                BiasofHiddenNeurons1 = normalize(BiasofHiddenNeurons1[None], norm='l2')
                InputWeight1=np.random.rand(NumberofHiddenNeurons,NumberofInputNeurons)*2-1
                InputWeight1 = normalize(InputWeight1, norm='l2')
                
                tempH=InputWeight1.dot(P);
                ind=np.ones((1,NumberofTrainingData));
                BiasMatrix=ind * BiasofHiddenNeurons1.T
                tempH=tempH+BiasMatrix;
            else:
                PP = np.where(PP > 1, 1, PP)
                PP = np.where(PP < -1, -1, PP)
                YYM_PP=PP
                PP1=asin(PP)
                PP1=PP1.real
                P=P_save
                InputWeight1=matrix_inverse(P, cuda = True, coefficient = C).T.dot(PP1.T)
                #InputWeight1=np.linalg.lstsq((np.identity(len(P))/C+P.dot(P.T)),P)[0].dot(PP1.T)
                InputWeight1 = InputWeight1.T
                fdafe=0
                tempH=InputWeight1.dot(P)
                PTV_P = np.concatenate((P,TV_P),axis=1)
                TTV_T = np.concatenate((saveT,TV_T),axis=1)
                YYM_H = InputWeight1.dot(PTV_P)
                BB1=np.array(PP1.shape)
                BB2=sum(sum(tempH-PP1))
                BBP=BB2/BB1[1]
                tempH = (tempH.T - BBP).T
                YYM_tempH = (YYM_H.T - BBP).T
                
            
            H = sin(tempH)
            
            if(j>1):
                YYM_H = sin(YYM_tempH)
                scaler2 = MinMaxScaler(feature_range=(-1,1))
                YYM_H = scaler2.fit_transform(YYM_H)
                H = YYM_H.T[0:NumberofTrainingData].T
                fname = name+str(chnl)+"feature_"+str(subnetwork)+".csv"
                tname = name+str(chnl)+"target.csv"
                np.savetxt(fname, YYM_H, delimiter=",")
                np.savetxt(tname, TTV_T, delimiter=",")
                
            P_save=P;
            P=H;
            FT=np.zeros((3,17766))
            E1=T
            TrainingAccuracy = []
            for i in range(1,3):
                a=0.0000001;
                Y2=E1;
                
                if(fdafe==0):
                    scaler = MinMaxScaler(feature_range=(-1,1))
                    Y22 = scaler.fit_transform(Y2) #scaler.inverse_transform(X)
                else:
                    scaler3 = MinMaxScaler(feature_range=(-1,1))
                    Y22 = scaler3.fit_transform(Y2)
                
                Y22 = np.where(Y22 > 1, 1, Y22)
                Y22 = np.where(Y22 < -1, -1, Y22)
                Y2=Y22                
                Y4=asin(Y2)
                Y4=Y4.real
                
                if(fdafe==0):
                    YYM = matrix_inverse(P.T, cuda = True, coefficient = C).dot(Y4.T)
                    #YYM = np.linalg.lstsq((np.identity(len(P))/C+linalg.dot(P_gpu,PT_gpu).get()),P)[0].dot(Y4.T)
                    YJX=YYM.T.dot(P).T;
                else:
                    PP = Y4.T.dot(matrix_inverse(YYM, cuda = True, coefficient = C)).T
                    #PP = Y4.T.dot(np.linalg.lstsq(  (np.identity(len(YYM))/C+YYM.dot(YYM.T)),YYM)[0].T).T
                    YJX=PP.T.dot(YYM);
                    
                BB1=np.array(Y4.shape);
                BB2=sum(YJX-Y4.T)
                BB=BB2/BB1[1];
                BB=BB[0]
                GXZ111 = (YJX.T - BB).T
                GXZ2=sin(GXZ111.T)
                
                FYY = scaler.inverse_transform(GXZ2).T
                FT1 =  FYY.T
                E1=E1-FT1
                if(i==1):
                    FT=FT1
                    fdafe=1
                else:
                    FT=FT+FT1
                
                if(subnetwork==1):
                    FTY=FT;
                    FTY_prev = FTY
                else:
                    FTY=FTY_prev-FT;
                    
                MissClassificationRate_Training=0;
                for idx in range(len(saveT[0])):
                    label_index_expected = np.where(saveT.T[idx] == np.amax(saveT.T[idx]))
                    label_index_actual = np.where(FTY.T[idx] == np.amax(FTY.T[idx]))
                    if(label_index_expected!=label_index_actual):
                        MissClassificationRate_Training+=1
                
                TrainingAccuracy.append(1-MissClassificationRate_Training/len(saveT[0]))
                 
            PP=PP+P;
        
        T=E1
        P=OrgP
        outputweight=YYM
        fdafe=0
    i=1

def featurecomb(name,sn,chnl):
  first=True
  for i in range(1,chnl+1):
    for j in range(1,sn+1):
      fname = name+str(i)+"feature_"+str(j)+".csv"
      if(first==True):
        Features = np.genfromtxt(fname, delimiter=',')
        first=False
      else:
        #Features =  maxpool(Features,np.genfromtxt(fname, delimiter=','))
        Features = Features + np.genfromtxt(fname, delimiter=',')
  return Features

def lastlayer(Features,Target,C2,No_train):
    kkk=1
    P = Features[:,0:No_train]
    T = Target[:,0:No_train]
    TV_P = Features[:,No_train:]
    TV_T = Target[:,No_train:]
    NumberofTrainingData=len(P[1])
    NumberofTestingData=len(TV_P[1])
    NumberofInputNeurons=len(P)
    E1=T
    
    Y2=E1
    scaler = MinMaxScaler(feature_range=(0.01,0.99))
    Y22 = scaler.fit_transform(Y2)
    scaler2 = MinMaxScaler(feature_range=(0.01,0.99))
    chumma = scaler2.fit_transform(TV_T)
    Y2=Y22.T
    def sigmoid(x):
    	# x = -x
        # return 1 / (1 + exp(-x))
        return 1 / (1 + math.exp(-x))
    sigmoid = np.vectorize(sigmoid)
    Y4=sigmoid(Y2).T
    YYM=matrix_inverse(P, cuda = True, coefficient = C).T.dot(Y4.T)
    #YYM=np.linalg.lstsq((np.identity(len(P))/C2+P.dot(P.T)),P)[0].dot(Y4.T)
    YJX=P.T.dot(YYM)
    BB1=np.array(Y4.shape)
    BB2=sum(YJX-Y4.T)
    BB=BB2[0]/BB1[1]
    GXZ111=(P.T.dot(YYM))-BB
    GXZ2=sigmoid(GXZ111).T
    FYY = scaler.inverse_transform(GXZ2)
    FT1=FYY
    TrainingAccuracy=math.sqrt(mse(E1,FT1))
    E1=E1-FT1
    Y=FYY
    D_YYM=YYM
    MissClassificationRate_Training=0
    for idx in range(len(T[0])):
        label_index_expected = np.where(T.T[idx] == np.amax(T.T[idx]))
        label_index_actual = np.where(Y.T[idx] == np.amax(Y.T[idx]))
        if(label_index_expected!=label_index_actual):
            MissClassificationRate_Training+=1
    TrainingAccuracy=(1-MissClassificationRate_Training/len(T[0]))*100
    print("TrainingAccuracy",TrainingAccuracy)
    
    GXZ1=D_YYM.T.dot(TV_P-BB)
    GXZ2=sigmoid(GXZ1).T
    FYY = scaler2.inverse_transform(GXZ2.T)
    TY2=FYY
    E1=TY2-TV_T
    TestingAccuracy=math.sqrt(mse(TY2,TV_T))
    MissClassificationRate_Testing=0
    for idx in range(len(TV_T[0])):
        label_index_expected = np.where(TV_T.T[idx] == np.amax(TV_T.T[idx]))
        label_index_actual = np.where(TY2.T[idx] == np.amax(TY2.T[idx]))
        if(label_index_expected!=label_index_actual):
            MissClassificationRate_Testing+=1
    TestingAccuracy=(1-MissClassificationRate_Testing/len(TV_T[0]))*100
    print("TestingAccuracy",TestingAccuracy)

In [ ]:
def loadata(traincsv,testcsv):
  train = np.genfromtxt(traincsv, delimiter=',')
  test = np.genfromtxt(testcsv, delimiter=',')
  T = train.T[0][None]
  P = train.T[1:]
  TV_T = test.T[0][None]
  TV_P = test.T[1:]
  NumberofTrainingData=len(P[1])
  NumberofTestingData=len(TV_P[1])
  NumberofInputNeurons=len(P);
  NumberofOutputNeurons=len(np.unique(T));        
  #ONE HOT ENCODING
  Temp_T = np.zeros((NumberofOutputNeurons,NumberofTrainingData))
  count = 0
  for i in T.T:
    Temp_T[int(i[0]-1)][count] = 1
    count+=1
  T = Temp_T*2-1
  count = 0
  Temp_T = np.zeros((NumberofOutputNeurons,NumberofTestingData))
  for i in TV_T.T:
    Temp_T[int(i[0]-1)][count] = 1
    count+=1
  TV_T = Temp_T*2-1
  return T, P, TV_T, TV_P, traincsv


In [ ]:
T, P, TV_T, TV_P, repeat = loadata('Scene_15_train.csv','Scene_15_test.csv')
NumberofHiddenNeurons=80
C=128
kkkk=2
sn=3
name='scene15_channel_'
chnl=1
load
for i in range(chnl):
  start_time = time.time()     
  subnetwrork(P,T,TV_P,TV_T, NumberofHiddenNeurons, C, kkkk, sn, name, i+1)
  print("Subspace Feature Extraction Time for channel_"+str(i+1)+" : ",time.time()-start_time,"\n")

start_time = time.time() 
Features = featurecomb(name, sn, chnl)
print("Combining Feature Time Taken : ",time.time()-start_time,"\n")

Target = np.genfromtxt(name+str(chnl)+"target.csv", delimiter=',')

C2=4096
No_train = len(P[1])
start_time = time.time()
lastlayer(Features,Target,C2,No_train)
print("Final Classification Time Taken : ",time.time()-start_time)

(3000, 1500) (15, 1500) (3000, 2985) (15, 2985)
Subspace Feature Extraction Time for channel_1 :  4.012771844863892 

Combining Feature Time Taken :  1.3897857666015625 

TrainingAccuracy 97.86666666666667
TestingAccuracy 93.96984924623115
Final Classification Time Taken :  0.2398059368133545


In [ ]:
NumberofHiddenNeurons=100
C=1
kkkk=2
sn=3
name='c101_channel_'
chnl=12
repeat=None;

for i in range(chnl):
  if(i<=2 and repeat!='c101_sift_train.csv'):
    T, P, TV_T, TV_P, repeat = loadata('c101_sift_train.csv','c101_sift_test.csv')
  elif(i>2 and i<=6 and repeat!='c101_HMP_train.csv'):
    T,P,TV_T,TV_P,repeat = loadata('c101_HMP_train.csv','c101_HMP_test.csv')
  elif(i>6 and repeat!='c101_Spatial_train.csv'):
    T,P,TV_T,TV_P,repeat = loadata('c101_Spatial_train.csv','c101_Spatial_test.csv')
  start_time = time.time()   
  subnetwrork(P,T,TV_P,TV_T, NumberofHiddenNeurons, C, kkkk, sn, name, i+1)
  print("Subspace Feature Extraction Time for channel_"+str(i+1)+" : ",time.time()-start_time,"\n")

start_time = time.time() 
Features = featurecomb(name, sn, chnl)
print("Combining Feature Time Taken : ",time.time()-start_time,"\n")

Target = np.genfromtxt(name+str(chnl)+"target.csv", delimiter=',')

C2=2^-13
No_train = len(P[1])
print(No_train)
start_time = time.time()
lastlayer(Features,Target,C2,No_train)
print("Final Classification Time Taken : ",time.time()-start_time)